In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date


C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#read in the data
df= pd.read_csv("N2 gauge calibration", sep='\t', index_col=False)
df.drop(columns=["Live comments","I_emission","I_grid","T-platinum","T-CERNOX","helium","DUAL.1"], inplace=True)
#df.drop(columns=['Threshold'], inplace=True)
#df.drop(labels=[0,1], axis=0)
df.head()


,Time,DUAL,Barion_2,Barion_1,I-injection
0,17/10/2022 14:14:16,0.19022,2.450000e-09,9.210000e-10,0.26361
1,17/10/2022 14:14:27,0.18987,2.460000e-09,9.240000e-10,0.26360
2,17/10/2022 14:14:37,0.19056,2.460000e-09,9.240000e-10,0.26360
3,17/10/2022 14:14:47,0.19031,2.460000e-09,9.280000e-10,0.26360
4,17/10/2022 14:14:58,0.19073,2.450000e-09,9.270000e-10,0.26360


In [3]:
#add elapsed time

timestamps=pd.to_datetime(df["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
df.insert(1,"Elapsed time",runtime)


In [4]:
df.head()

,Time,Elapsed time,DUAL,Barion_2,Barion_1,I-injection
0,17/10/2022 14:14:16,0.0,0.19022,2.450000e-09,9.210000e-10,0.26361
1,17/10/2022 14:14:27,11.0,0.18987,2.460000e-09,9.240000e-10,0.26360
2,17/10/2022 14:14:37,21.0,0.19056,2.460000e-09,9.240000e-10,0.26360
3,17/10/2022 14:14:47,31.0,0.19031,2.460000e-09,9.280000e-10,0.26360
4,17/10/2022 14:14:58,42.0,0.19073,2.450000e-09,9.270000e-10,0.26360


In [5]:
#Plotting the general pressure evolution for overview
plt.figure()
plt.plot((df["Elapsed time"]/3600),df['Barion_1'],marker=".", markersize=5,label='BA1')
plt.plot((df["Elapsed time"]/3600),df['Barion_2'],marker=".", markersize=5,label='BA2')
plt.plot((df["Elapsed time"]/3600),df['DUAL'],marker=".", markersize=5,label='110 F-R')
plt.plot((df["Elapsed time"]/3600),df['I-injection'],marker=".", markersize=5,label='1.1 F-R')
plt.xlabel('Elapsed time (h)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Pressure evolution")
plt.yscale('log')

In [6]:
#Disregarding all unnecesessary data before injection
x=0.371*3600
print("Value of Elapsed time at the first relevant datapoint",x)
#find the index of this value
print(df.loc[df["Elapsed time"]==1341.0])

Value of Elapsed time at the first relevant datapoint 1335.6
                    Time  Elapsed time     DUAL      Barion_2      Barion_1  \
130  17/10/2022 14:36:37        1341.0  0.79494  2.370000e-09  9.050000e-10   

     I-injection  
130      0.86938  


In [7]:
#New sub-dataset with relevant data
df2 = df.iloc[130: , :]
df2.tail()


,Time,Elapsed time,DUAL,Barion_2,Barion_1,I-injection
213,17/10/2022 14:50:52,2196.0,-0.065303,5.940000e-08,1.320000e-08,0.000425
214,17/10/2022 14:51:03,2207.0,-0.064710,5.890000e-08,1.310000e-08,0.000395
215,17/10/2022 14:51:13,2217.0,-0.069347,5.850000e-08,1.310000e-08,0.000420
216,17/10/2022 14:51:23,2227.0,-0.065154,5.800000e-08,1.300000e-08,0.000424
217,17/10/2022 14:51:33,2237.0,-0.064883,5.000000e-08,1.150000e-08,0.000424


In [8]:
#New graph
plt.figure()
plt.plot((df2["Elapsed time"]/3600),df2['Barion_1'],marker=".", markersize=5,label='BA1')
plt.plot((df2["Elapsed time"]/3600),df2['Barion_2'],marker=".", markersize=5,label='BA2')
plt.plot((df2["Elapsed time"]/3600),df2['DUAL'],marker=".", markersize=5,label='110 F-R')
plt.plot((df2["Elapsed time"]/3600),df2['I-injection'],marker=".", markersize=5,label='1.1 F-R')
plt.xlabel('Elapsed time (h)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Pressure evolution during injection")
plt.yscale('log')

## Procedure for the calculation
#### Assumptions
1) Barion gauges' manufacturer made calibrations are valid <br/>
2) The EGA support bearing has minimal conductivity <br/>
3) The distribution of gas molecules in the vessel in homogeneous <br/>
4) Conductance of the calibrated orifice is the limiting factor for achieving pressure equilibrium between the injection volume and experimental chamber <br/>
4) Use Barion_1 gauge as the closest to the turbo and orifice

In [9]:
# Conductance (N2) for the calibrated orifice [l/s]
c_orif=10.13
#Bolzman constant [m2 kg s^-2 K^-1] 
k_B=1.38e-23
#Temperature [K]
T=273+21
#Time amount for achieving equilibrium + pressure delta
start=1e-05
df2_start = df2.iloc[(df2['Barion_1']-start).abs().argsort()[:2]]
print("start index",df2_start.index.tolist())
stop=1.6522e-08
df2_stop=df2.iloc[(df2["Barion_1"]-stop).abs().argsort()[:2]]
print("stop index",df2_stop.index.tolist())

start index [173, 165]
stop index [195, 196]


In [10]:
#inj dataframe where pressure decrease is a straight line on the graph
inj=df.iloc[171:183]
inj.head()
#plotting
plt.figure()
plt.plot((inj["Elapsed time"]),inj['Barion_1'],marker=".", markersize=5,label='BA1')
plt.plot((inj["Elapsed time"]),inj['I-injection'],marker=".", markersize=5,label='1.1 f-r')
plt.plot((inj["Elapsed time"]),inj['Barion_2'],marker=".", markersize=5,label='BA2')


plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Injection, linear part (logarithmic y scale)")
plt.yscale('log')


In [11]:

print("Value of Elapsed time at the first relevant datapoint",x)
#find the index of this value
print(inj.loc[inj["Elapsed time"]==1763.0])


Value of Elapsed time at the first relevant datapoint 1335.6
                    Time  Elapsed time     DUAL  Barion_2  Barion_1  \
171  17/10/2022 14:43:39        1763.0  0.26669   0.00009  0.000021   

     I-injection  
171      0.24699  


In [12]:
z=1886.0
inj.iloc[(inj['Elapsed time'] - z).abs().argsort(),:]


,Time,Elapsed time,DUAL,Barion_2,Barion_1,I-injection
182,17/10/2022 14:45:33,1877.0,-0.061062,0.000003,5.790000e-07,0.007941
181,17/10/2022 14:45:22,1866.0,-0.052458,0.000004,7.890000e-07,0.010804
180,17/10/2022 14:45:12,1856.0,-0.037935,0.000005,1.090000e-06,0.014614
179,17/10/2022 14:45:02,1846.0,-0.033227,0.000007,1.490000e-06,0.019935
178,17/10/2022 14:44:51,1835.0,-0.028640,0.000009,2.050000e-06,0.027395
177,17/10/2022 14:44:41,1825.0,-0.017622,0.000013,2.810000e-06,0.037500
176,17/10/2022 14:44:31,1815.0,-0.001398,0.000018,3.850000e-06,0.051692
175,17/10/2022 14:44:20,1804.0,0.030801,0.000024,5.080000e-06,0.071281
174,17/10/2022 14:44:10,1794.0,0.050229,0.000032,7.070000e-06,0.096311
173,17/10/2022 14:44:00,1784.0,0.090760,0.000044,9.570000e-06,0.132270


In [14]:
V=10.13*3.92e-5/(0.05/9)
print(V)

0.07147727999999999


In [13]:
#delta p
p_max=inj["Barion_1"][171]
p_min=inj["Barion_1"][183]
delta_p=p_max-p_min
print(delta_p)
#delta t
delta_t=inj["Elapsed time"][183]-inj["Elapsed time"][171]
dp_dt=delta_p/delta_t
print(dp_dt)

KeyError: 183

### Formula for calculating injection volume
$$V[l]\cdot \frac{dp}{dt}[\frac{mbar}{s}]=C_{orif}[\frac{l}{s}]\cdot \Delta(Ba_2-Ba_1)[mbar]$$


In [59]:
#V*dp/dt=C(delta(Ba2-Ba1))
#algorithm: for every two consecutive datapoints, take the delta for I-injection and the two relative timestamps
#and the difference of Ba2-Ba1
inj["Time diff"]=inj["Elapsed time"].diff()
inj["p-dif fr"]=abs(inj["I-injection"].diff())
#baryon gauges
inj["p-dif baryon"]=inj["Barion_2"]-inj["Barion_1"]
print(inj["Barion_2"].rolling(2).sum()/2)
#slope=((inj["I-injection"]/inj["Elapsed time"].tolist()))
#p_dif=((inj["Barion_2"]-inj["Barion_1"].tolist()))
#V=c_orif*p_dif/slope
#print(V)
#av_V=np.mean(V)
#print("Average injection volume over all datapoints in the linear area",av_V,"l")

171         NaN
172    0.000074
173    0.000050
174    0.000038
175    0.000028
176    0.000021
177    0.000015
178    0.000011
179    0.000008
180    0.000006
181    0.000004
182    0.000003
Name: Barion_2, dtype: float64


C:\Users\etiirine\AppData\Local\Temp\ipykernel_3016\1367578026.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inj["Time diff"]=inj["Elapsed time"].diff()
C:\Users\etiirine\AppData\Local\Temp\ipykernel_3016\1367578026.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inj["p-dif fr"]=abs(inj["I-injection"].diff())
C:\Users\etiirine\AppData\Local\Temp\ipykernel_3016\1367578026.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [60]:
inj.head()

,Time,Elapsed time,DUAL,Barion_2,Barion_1,I-injection,Time diff,p-dif fr,p-dif baryon,Inj_V
171,17/10/2022 14:43:39,1763.0,0.266690,0.000090,0.000021,0.246990,NaN,NaN,0.000069,NaN
172,17/10/2022 14:43:50,1774.0,0.162500,0.000057,0.000013,0.180410,11.0,0.066580,0.000044,0.094978
173,17/10/2022 14:44:00,1784.0,0.090760,0.000044,0.000010,0.132270,10.0,0.048140,0.000034,0.082414
174,17/10/2022 14:44:10,1794.0,0.050229,0.000032,0.000007,0.096311,10.0,0.035959,0.000025,0.083611
175,17/10/2022 14:44:20,1804.0,0.030801,0.000024,0.000005,0.071281,10.0,0.025030,0.000019,0.089745


In [61]:
inj["Inj_V"]=10.13*inj["p-dif baryon"]/(inj["p-dif fr"]/inj["Time diff"])

C:\Users\etiirine\AppData\Local\Temp\ipykernel_3016\2830528240.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inj["Inj_V"]=10.13*inj["p-dif baryon"]/(inj["p-dif fr"]/inj["Time diff"])


In [53]:
inj.head()

,Time,Elapsed time,DUAL,Barion_2,Barion_1,I-injection,Time diff,p-dif fr,p-dif baryon,Inj_V
171,17/10/2022 14:43:39,1763.0,0.266690,0.000090,0.000021,0.246990,NaN,NaN,NaN,NaN
172,17/10/2022 14:43:50,1774.0,0.162500,0.000057,0.000013,0.180410,11.0,0.066580,0.000057,0.094978
173,17/10/2022 14:44:00,1784.0,0.090760,0.000044,0.000010,0.132270,10.0,0.048140,0.000039,0.082414
174,17/10/2022 14:44:10,1794.0,0.050229,0.000032,0.000007,0.096311,10.0,0.035959,0.000030,0.083611
175,17/10/2022 14:44:20,1804.0,0.030801,0.000024,0.000005,0.071281,10.0,0.025030,0.000022,0.089745


In [62]:
avg=inj["Inj_V"].mean(skipna=True)
print("Average inj volume with valve 2 open and not accounting for residual signal",avg,"l")
V_nupro=(4.3/2)/1000
V_cmr=4.2/1000
V_ref=V_nupro/2+V_cmr
print("True injection volume:",avg-V_ref,"l")

Average inj volume with valve 2 open and not accounting for residual signal 0.0755076257500205 l
True injection volume: 0.0702326257500205 l
